$$-\Delta u =f$$



In [15]:
from petsc4py import PETSc

ImportError: dlopen(/opt/anaconda3/lib/python3.12/site-packages/petsc4py/lib/PETSc.cpython-312-darwin.so, 0x0002): Library not loaded: @rpath/liblapack.3.dylib
  Referenced from: <76FAFCCD-D660-32D5-8C7D-C7163B299263> /opt/anaconda3/lib/libpetsc.3.21.5.dylib
  Reason: tried: '/opt/anaconda3/lib/liblapack.3.dylib' (no such file), '/opt/anaconda3/lib/python3.12/site-packages/petsc4py/lib/../../../../liblapack.3.dylib' (no such file), '/opt/anaconda3/lib/python3.12/site-packages/petsc4py/lib/../../../../liblapack.3.dylib' (no such file), '/opt/anaconda3/bin/../lib/liblapack.3.dylib' (no such file), '/opt/anaconda3/bin/../lib/liblapack.3.dylib' (no such file), '/usr/local/lib/liblapack.3.dylib' (no such file), '/usr/lib/liblapack.3.dylib' (no such file, not in dyld cache)

In [7]:


m, n = 10, 12

hx = 1/(m-1)
hy = 1/(n-1)

# 
A = PETSc.Mat()
A.create(PETSc.COMM_WORLD) # Puede servir para progración en paralelo
A.setSizes([m*n,m*n])
A.setType('aij') # Dispersa (Sparse)


# llenar los valores:A

diagv = 2/hx**2 +2/hy**2
offdx = -1.0/hx**2
offdy = -1.0/hy**2

# loop over owned block of rows on this
# processor and insert entry values
Istart, Iend = A.getOwnershipRange()
for I in range(Istart, Iend) :
    A[I,I] = diagv
    i = I//n # map row number to
    j = I - i*n # grid coordinates
    if i> 0 : J = I-n; A[I,J] = offdx
    if i< m-1: J = I+n; A[I,J] = offdx
    if j> 0 : J = I-1; A[I,J] = offdy
    if j< n-1: J = I+1; A[I,J] = offdy

A.assemblyBegin()
A.assemblyEnd()

In [14]:
# Crear el solucionador
ksp = PETSc.KSP()
ksp.create(PETSc.COMM_WORLD)

ksp.setType('cg') #gradiente conjugado

# precondicionadores
ksp.getPC().setType('icc')

#  A 
#  
#

#PA = Pb

# Crear la matriz A (ejemplo de una matriz dispersa simétrica y definida positiva)
n = 10  # tamaño del sistema
A = PETSc.Mat().createAIJ([n, n])
A.setFromOptions()
A.setUp()

for i in range(n):
    A.setValue(i, i, 2.0)  # valores diagonales
    if i > 0:
        A.setValue(i, i - 1, -1.0)  # valores subdiagonales
        A.setValue(i - 1, i, -1.0)  # valores superdiagonales
A.assemble()

# Establecer la matriz en el solucionador
ksp.setOperators(A)

# Crear los vectores x y b
x, b = A.getVecs()
x.set(0)  # vector nulo como punto de partida
b.set(1)  # b lleno de unos

# Resolver el sistema
ksp.solve(b, x)    #PA x= Pb #P=I 

# Imprimir la solución
x_array = x.getArray()
print("Solución x:")
print(x_array)


Solución x:
[ 5.  9. 12. 14. 15. 15. 14. 12.  9.  5.]


In [38]:
from petsc4py import PETSc
snes = PETSc.SNES().create()

comm = PETSc.COMM_WORLD

# Crear el vector para la solución y para la función no lineal
x = PETSc.Vec().create(comm=comm)
x.setSizes(1)
x.setFromOptions()
f = x.duplicate()

y = x
# Definir la función no lineal f(x) = x^2 - 4
def eval_f(snes, x, f):
    x_val = x.getValue(0)   # Obtener el valor de x sin array
    f_val = x_val ** 2 - 4   # (MODIFICAR) Evaluar f(x) = x^2 - 4
    f.setValue(0, f_val)     # Asignar f(x) sin array

# Definir la función Jacobiana J(x) = d(f)/dx = 2 * x
def eval_jacobian(snes, x, J, P):
    x_val = x.getValue(0)      # Obtener el valor de x
    J.setValue(0, 0, 2 * x_val) # (MODIAFICAR) Derivada de f con respecto a x
    J.assemble()

# Configurar la función y el Jacobiano en el solver SNES
snes.setFunction(eval_f, f)
J = PETSc.Mat().createAIJ([1, 1], comm=comm)
J.setUp()
snes.setJacobian(eval_jacobian, J)
snes.setTolerances(atol=1e-5, max_it=1000)
# Inicializar el valor inicial de x

# Resolver para x cercano a -2
x.setValue(0, -1)
snes.solve(None, x)
print(f"Solución cercana a -2: x = {x.getValue(0)}")

# Resolver para x cercano a 2
x.setValue(0, 1)
snes.solve(None, x)
print(f"Solución cercana a 2: x = {x.getValue(0)}")




#-----(-2) -x---- |------(x)------(2)
# x = 2, x=-2


Solución cercana a -2: x = -2.0000000929222947
Solución cercana a 2: x = 2.0000000929222947


In [20]:
help(x.setValue)

Help on built-in function setValue:

setValue(...) method of petsc4py.PETSc.Vec instance
    Vec.setValue(self, index: int, value: Scalar, addv: InsertModeSpec = None) -> None
    Insert or add a single value in the vector.
    
            Not collective.
    
            Parameters
            ----------
            index
                Location to write to. Negative indices are ignored.
            value
                Value to insert at ``index``.
            addv
                Insertion mode.
    
            Notes
            -----
            The values may be cached so `assemblyBegin` and `assemblyEnd`
            must be called after all calls of this method are completed.
    
            Multiple calls to `setValue` cannot be made with different values
            for ``addv`` without intermediate calls to `assemblyBegin` and
            `assemblyEnd`.
    
            See Also
            --------
            setValues, petsc.VecSetValues
    
            
    Source co